[![Cython](cython.png)](http://cython.org)



* Built to be a Python+C language for high performance computations
* Performance computation space in competition with Numba
* Due to design, also makes binding easy
* Easy to customize result
* Can write Python 2 or 3, regardless of calling language

Downsides:
* Requires learning a new(ish) language
* Have to think with three hats
* *Very* verbose

# Aside: Speed comparison Python, Cython, [Numba](https://numba.pydata.org)

In [ ]:
def f(x):
    for _ in range(100000000):
        x=x+1
    return x

In [ ]:
%%time
f(1)

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def f(int x):
    for _ in range(10000000):
        x=x+1
    return x

In [ ]:
%%timeit
f(23)

In [ ]:
import numba
@numba.jit
def f(x):
    for _ in range(10000000):
        x=x+1
    return x

In [ ]:
%time
f(41)

In [ ]:
%%timeit
f(41)

# Binding with [Cython](https://cython.org)

In [ ]:
%%writefile simpleclass.pxd
# distutils: language = c++

cdef extern from "SimpleClass.hpp":
    cdef cppclass Simple:
        Simple(int x)
        int get()

In [ ]:
%%writefile cythonclass.pyx
# distutils: language = c++

from simpleclass cimport Simple as cSimple

cdef class Simple:
    cdef cSimple *cself
    
    def __cinit__(self, int x):
        self.cself = new cSimple(x)
    
    def get(self):
        return self.cself.get()
    
    def __dealloc__(self):
        del self.cself

In [ ]:
!cythonize cythonclass.pyx

In [ ]:
!g++ cythonclass.cpp -shared {inc} -o cythonclass.so {plat}

In [ ]:
import cythonclass
x = cythonclass.Simple(3)
x.get()